# Photonic circuit simulations

At the scale of an entire chip, full electromagnetic simulations become prohibitive. To gain insight on the behaviour of an entire circuit, circuit simulations using **compact models** can instead be performed.

In the semiconductor industry, a compact model is a "computationally efficient description of the terminal properties of a device as a function of terminal voltages" [(Sivakumar Mudanai (Intel), on Nanohub)](https://nanohub.org/resources/21459/download/2014.06.10-Mudanai-NEEDS.pdf). These computationally efficient models can then be cascaded to obtain full circuit behaviour.

We can generalize the concept to photonics : a compact model would be an efficient description of the properties of a device as a function of a few parameters such as some geometrical properties (gap of a coupler, radius of a ring resonator, etc.), wavelength, voltage applied, etc. 

# Ex: compact model for a waveguide

Recall how to obtain this as done in *02-MPB_dispersion.ipynb* :

![alt text](http://www.simpetus.com/files/SOI_waveguide_bands.png)

*Source : http://www.simpetus.com/files/SOI_waveguide_bands.png*

Remember the general phase velocity $ v_p = \frac{\omega}{k_x}  $ (since $x$ is propagation direction here), and define refractive index as usual $n = \frac{c}{v_p}$. But now, we have a dispersion $\omega(k_x)$ which is determined given a single $\omega(\lambda) = 2\pi f = 2\pi c/\lambda$. **Hence, the ratio $v_p$ changes with (free-space) wavelength, which can be captured by an effective index $n_{eff} = c \left( \frac{k_x \left( \omega (\lambda) \right)}{\omega (\lambda)} \right) $.

So for *e.g.* the fundamental mode of a 500-nm rib waveguide, we can get a plot like this :

![wvg_compact](images/rib_wvg_compact.PNG)

*Source : Chrostowski, L., & Hochberg, M. (2015). Silicon Photonics Design: From Devices to Systems (pp. 313-348). Cambridge: Cambridge University Press.*

Leading to (in this case) a compact model parametrized by a few-parameter polynomial fit :

$$ n_{eff}(\lambda) = 2.57 - 0.85\left( \lambda[\mu m] - 1.55 \right) $$

From this, we can compute group index, which can be useful to compute *e.g.* free spectral range of resonators. But the point it that we went from having to solve a 2D eigenproblem for every waveguide to a simple linear equation! This is useful when computing properties of large systems.

# In general : S-parameters

For a general N-port device, the properties we are most often interested in are the elements of the scattering matrix (S-parameters). The concept originated in microwave design, so a good introduction is contained in Ch. 4.3 of [1]. As usual, the main reference for the class [2] has a good chapter on it (Ch. 9).

S-parameters relate the complex amplitudes of incoming and outgoing modes of your device :

![n-ports](images/N-port.png)

\begin{equation}
\begin{bmatrix}
b_1\\ 
b_2\\ 
\vdots \\ 
b_N
\end{bmatrix} = \begin{bmatrix}
S_{11} & S_{12} & ... & S_{1N} \\ 
S_{21} & S_{22} & ...  & S_{2N} \\ 
\vdots & \vdots  & \ddots & \vdots \\ 
S_{N1} & S_{N2} & ... & S_{NN}
\end{bmatrix} \begin{bmatrix}
a_1\\ 
a_2\\ 
\vdots \\ 
a_N
\end{bmatrix}
\end{equation}

Hence, by exciting mode $a_j$ and monitoring the outgoing complex amplitude at port $b_i$ (with all other incoming modes $a_k = 0$ for $k\neq j$), the coefficient $S_{ij}$ can be determined via $S_{ij} = \frac{b_i}{a_j}$. **The concept of "mode" here is abstract, and can represent different physical ports, or different orthogonal spatial modes in the same physical port.** The coefficients $S_{ij}$ can be computed or measured as a function of a few parameters (geometry, wavelength, etc.), yielding a tensor in practice.

You already know how to extract S-parameters from FDTD simulations; see notebook *06-MEEP_S_parameters.ipynb*.

## Some properties

* Cascading devices : Output of one device is input to the other, hence $\mathbf{T} = \prod_i \mathbf{T}_i$ with $T$ the transfer matrix related to scattering matrix; see Ref. [5].
* Passive components are reciprocal $S_{ij} = S_{ji}$

# Software

Simply multiplying transfer matrices is fairly easy. However, sometimes you may want to consider active devices such as semiconductor optical amplifiers with time-dynamics. In that case, you would need to modify the S-scattering model above to a state-based model :

$$ b = S\cdot a + f(t, a(t), a(t-\Delta t),\dots) $$

where the outgoing amplitude can now depend nonlineary on past inputs or internal states. You may then consider using using specialized software for convenience instead of writing your own ODE solvers.

## Commercial

Again, [Lumerical](https://www.lumerical.com/), and [Synopsys](https://www.synopsys.com/photonic-solutions.html), have fairly complete circuit-level solvers. With some work, you can integrate them with the device-level solvers. In addition, there are others : [Aspic](http://www.aspicdesign.com/), [Luceda Caphe](https://www.lucedaphotonics.com/en/product/caphe-photonic-circuit-simulator-ipkissflow), [VPI Photonics](https://www.vpiphotonics.com/index.php), etc. These again mostly offer convenience by implementating all the above natively, offering graphical user interfaces, component libraries, etc.

## Open-source options?

There are not many openly-accessible packages to facilitate cascading many circuit elements. I was gracefully offered a preview copy of [Photontorch](https://photontorch.com/) by the designer Floris Laporte.

**June 2020 update : PhotonTorch is no longer available as an open-source option. Consider using something like [Simphony](https://github.com/BYUCamachoLab/simphony) instead.**

 # What is Photontorch


"Photontorch is a photonic simulation tool for highly parallel simulation and optimization of photonic circuits in time and frequency domain. Photontorch features CUDA enabled simulation and optimization of photonic circuits. It leverages the deep learning framework PyTorch to view the photonic circuit as essentially a sparsely connected recurrent neural network. This enables the use of native PyTorch optimizers to optimize the (physical) parameters of your circuit."

Built-in tools facilitate the defining of components and their networking, which is what we are principally interested in. Feel free to explore the PyTorch-enhanced optimization functionalities!

# References

1. Pozar, D. M. (1990). Microwave Engineering
2. Chrostowski, L., & Hochberg, M. (2015). Photonic circuit modelling. In Silicon Photonics Design: From Devices to Systems (pp. 313-348). Cambridge: Cambridge University Press. 
3. Laporte, F., Dambre, J. & Bienstman, P. Highly parallel simulation and optimization of photonic circuits in time and frequency domain based on the deep-learning framework PyTorch. Sci Rep9, 5918 (2019). https://doi.org/10.1038/s41598-019-42408-2
4. https://photontorch.com/
5. O. Schwelb, "Generalized analysis for a class of linear interferometric networks. I. Analysis," in IEEE Transactions on Microwave Theory and Techniques, vol. 46, no. 10, pp. 1399-1408, Oct. 1998.